# Create IPyParallel Profile

Borrowed from https://gist.github.com/basnijholt/c375ea2d1df6702492b619e0873d6c7c

Overtime this should be automated with qhub-hpc and provde better slurm submission scripts

In [6]:
! ipython profile create --parallel --profile=slurm

In [4]:
%%writefile /home/vagrant/.ipython/profile_slurm/ipcontroller_config.py

c.HubFactory.ip = u'*'
c.HubFactory.registration_timeout = 600

Overwriting /home/vagrant/.ipython/profile_slurm/ipcontroller_config.py


In [5]:
%%writefile /home/vagrant/.ipython/profile_slurm/ipengine_config.py

c.IPEngineApp.wait_for_url_file = 300
c.EngineFactory.timeout = 300

Overwriting /home/vagrant/.ipython/profile_slurm/ipengine_config.py


In [19]:
%%writefile /home/vagrant/.ipython/profile_slurm/ipcluster_config.py

c.IPClusterStart.controller_launcher_class = 'SlurmControllerLauncher'
c.IPClusterEngines.engine_launcher_class = 'SlurmEngineSetLauncher'

c.SlurmEngineSetLauncher.batch_template = """#!/bin/sh
#SBATCH --ntasks={n}
#SBATCH --job-name=ipy-engine-
srun ipengine --profile-dir="{profile_dir}" --cluster-id=""
"""

Overwriting /home/vagrant/.ipython/profile_slurm/ipcluster_config.py


# Create IParallel Cluster

This following cell is blocking and should not be run. Prefereably you should visit `/user/<username>/tree#ipyclusters` and create cluster there.

In [25]:
# ! ipcluster start --n=1 --profile=slurm

Check that the cluster was indeed created. You should see a `ipy-controller` and `ipy-engine`.

In [28]:
! squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                 4   general spawner-  vagrant  R      19:26      1 hpc02-test
                13   general ipy-cont  vagrant  R       0:28      1 hpc03-test
                14   general ipy-engi  vagrant  R       0:25      1 hpc04-test


# Using IPyCluster

Read docs for usage https://ipyparallel.readthedocs.io/en/latest/

In [32]:
import ipyparallel
print(ipyparallel.__version__)

client = ipyparallel.Client(profile='slurm')

6.3.0


In [33]:
client.ids

[0]

In [48]:
def slow_power(x, i=5):
    import time
    time.sleep(2)
    return x**i

In [49]:
result = client[:].apply(slow_power, 5, 5)

In [50]:
%time result.result()

CPU times: user 2.7 ms, sys: 539 µs, total: 3.23 ms
Wall time: 1.72 s


[3125]